This notebook computes the word mover's distance between a query (profile) and a document (letter). experiment a) uses queries and documents as they are, experiment b) removes semantically identical phrases from the documents before WMD computation. Results remain between 0 and 1, i.e. a null result (given that maximum values for WMD goes to infinity)

In [ ]:
queries = [
    "no checking account,12,critical account/ other credits existing (not at this bank),radio/television,976,unknown/ no savings account,.. >= 7 years,4,male   : single,none,4,car or other,35,none,own,2,skilled employee / official,1,none,yes,1", #profile 805, letter 1
    "no checking account,12,existing credits paid back duly till now,radio/television,886,unknown/ no savings account,1  <= ... < 4 years,4,female : divorced/separated/married,none,2,car or other,21,none,own,1,skilled employee / official,1,none,yes,1", #letter 2
    "0 <= ... <  200 DM,6,existing credits paid back duly till now,radio/television,484,... <  100 DM,4  <= ... < 7 years,3,male   : married/widowed,guarantor,3,real estate,28,bank,own,1,unskilled - resident,1,none,yes,1", #letter 3
    "... <    0 DM,48,critical account/ other credits existing (not at this bank),car (used),6143,... <  100 DM,.. >= 7 years,4,female : divorced/separated/married,none,4,unknown / no property,58,stores,for free,2,unskilled - resident,1,none,yes,2",
    "no checking account,24,critical account/ other credits existing (not at this bank),car (used),3868,... <  100 DM,.. >= 7 years,4,female : divorced/separated/married,none,2,car or other,41,none,rent,2,management/ self-employed/ highly qualified employee/ officer,1,yes registered under the customers name,yes,1",
    "no checking account,15,delay in paying off in the past,radio/television,1478,... <  100 DM,1  <= ... < 4 years,4,male   : married/widowed,none,3,real estate,33,bank,own,2,skilled employee / official,1,none,yes,1",
    "no checking account,48,existing credits paid back duly till now,business,4844,... <  100 DM,unemployed,3,male   : single,none,2,car or other,33,bank,rent,1,management/ self-employed/ highly qualified employee/ officer,1,yes registered under the customers name,yes,2",
    "... <    0 DM,9,existing credits paid back duly till now,furniture/equipment,2136,... <  100 DM,1  <= ... < 4 years,3,male   : single,none,2,real estate,25,none,own,1,skilled employee / official,1,none,yes,1",
    "no checking account,15,existing credits paid back duly till now,car (new),3556,unknown/ no savings account,1  <= ... < 4 years,3,male   : single,none,2,unknown / no property,29,none,own,1,skilled employee / official,1,none,yes,1",
    "no checking account,21,delay in paying off in the past,furniture/equipment,1591,100 <= ... <  500 DM,4  <= ... < 7 years,4,male   : single,none,3,real estate,34,none,own,2,management/ self-employed/ highly qualified employee/ officer,1,none,yes,1",
    "... <    0 DM,45,no credits taken/ all credits paid back duly,business,11816,... <  100 DM,.. >= 7 years,2,male   : single,none,4,car or other,29,none,rent,2,skilled employee / official,1,none,yes,2",
    "0 <= ... <  200 DM,54,no credits taken/ all credits paid back duly,business,15945,... <  100 DM,... < 1 year,3,male   : single,none,4,unknown / no property,58,none,rent,1,skilled employee / official,1,yes registered under the customers name,yes,2",
    "... <    0 DM,18,existing credits paid back duly till now,furniture/equipment,1131,... <  100 DM,unemployed,4,female : divorced/separated/married,none,2,car or other,33,none,own,1,skilled employee / official,1,none,yes,2",
    "no checking account,15,existing credits paid back duly till now,car (new),5324,500 <= ... < 1000 DM,.. >= 7 years,1,female : divorced/separated/married,none,4,unknown / no property,35,none,for free,1,skilled employee / official,1,none,yes,1",
    "... <    0 DM,12,no credits taken/ all credits paid back duly,car (new),1082,... <  100 DM,1  <= ... < 4 years,4,male   : single,none,4,car or other,48,bank,own,2,skilled employee / official,1,none,yes,2",
    "no checking account,10,critical account/ other credits existing (not at this bank),furniture/equipment,2069,unknown/ no savings account,1  <= ... < 4 years,2,male   : married/widowed,none,1,car or other,26,none,own,2,skilled employee / official,1,none,no,1",
    "no checking account,4,critical account/ other credits existing (not at this bank),car (new),3380,... <  100 DM,4  <= ... < 7 years,1,female : divorced/separated/married,none,1,real estate,37,none,own,1,skilled employee / official,2,none,yes,1",
    "... <    0 DM,18,delay in paying off in the past,education,8471,unknown/ no savings account,1  <= ... < 4 years,1,female : divorced/separated/married,none,2,car or other,23,none,rent,2,skilled employee / official,1,yes registered under the customers name,yes,1",
    "... <    0 DM,20,critical account/ other credits existing (not at this bank),furniture/equipment,4272,... <  100 DM,.. >= 7 years,1,female : divorced/separated/married,none,4,building society savings agreement/ life insurance,24,none,own,2,skilled employee / official,1,none,yes,1",
    "no checking account,12,critical account/ other credits existing (not at this bank),radio/television,2331,unknown/ no savings account,.. >= 7 years,1,male   : single,co-applicant,4,real estate,49,none,own,1,skilled employee / official,1,yes registered under the customers name,yes,1"
    "... <    0 DM,24,existing credits paid back duly till now,furniture/equipment,2996,unknown/ no savings account,1  <= ... < 4 years,2,male   : married/widowed,none,4,car or other,20,none,own,1,skilled employee / official,1,none,yes,2",
    "0 <= ... <  200 DM,24,existing credits paid back duly till now,furniture/equipment,3069,100 <= ... <  500 DM,.. >= 7 years,4,male   : single,none,4,unknown / no property,30,none,for free,1,skilled employee / official,1,none,yes,1",
    "... <    0 DM,36,existing credits paid back duly till now,furniture/equipment,5179,... <  100 DM,4  <= ... < 7 years,4,male   : single,none,2,building society savings agreement/ life insurance,29,none,own,1,skilled employee / official,1,none,yes,2",
    "no checking account,12,existing credits paid back duly till now,car (new),640,... <  100 DM,1  <= ... < 4 years,4,male   : divorced/separated,none,2,real estate,49,none,own,1,unskilled - resident,1,none,yes,1",
    "0 <= ... <  200 DM,36,existing credits paid back duly till now,furniture/equipment,9034,100 <= ... <  500 DM,... < 1 year,4,male   : single,co-applicant,1,unknown / no property,29,none,rent,1,management/ self-employed/ highly qualified employee/ officer,1,yes registered under the customers name,yes,2",
    "0 <= ... <  200 DM,21,existing credits paid back duly till now,business,2767,100 <= ... <  500 DM,.. >= 7 years,4,male   : divorced/separated,none,2,car or other,61,bank,rent,2,unskilled - resident,1,none,yes,2",
    "0 <= ... <  200 DM,18,existing credits paid back duly till now,car (new),1042,unknown/ no savings account,1  <= ... < 4 years,4,female : divorced/separated/married,none,2,building society savings agreement/ life insurance,33,none,own,1,skilled employee / official,1,none,yes,2",
    "... <    0 DM,18,critical account/ other credits existing (not at this bank),furniture/equipment,1049,... <  100 DM,... < 1 year,4,female : divorced/separated/married,none,4,building society savings agreement/ life insurance,21,none,rent,1,skilled employee / official,1,none,yes,1",
    "... <    0 DM,36,existing credits paid back duly till now,others,15857,... <  100 DM,unemployed,2,male   : divorced/separated,co-applicant,3,car or other,43,none,own,1,management/ self-employed/ highly qualified employee/ officer,1,none,yes,1",
    "no checking account,15,existing credits paid back duly till now,domestic appliances,874,unknown/ no savings account,... < 1 year,4,female : divorced/separated/married,none,1,real estate,24,none,own,1,skilled employee / official,1,none,yes,1",
    "0 <= ... <  200 DM,60,existing credits paid back duly till now,car (new),14027,... <  100 DM,4  <= ... < 7 years,4,male   : single,none,2,unknown / no property,27,none,own,1,management/ self-employed/ highly qualified employee/ officer,1,yes registered under the customers name,yes,2",
    "no checking account,15,critical account/ other credits existing (not at this bank),furniture/equipment,2788,... <  100 DM,4  <= ... < 7 years,2,female : divorced/separated/married,co-applicant,3,car or other,24,bank,own,2,skilled employee / official,1,none,yes,1",
    "no checking account,18,existing credits paid back duly till now,car (new),2662,unknown/ no savings account,4  <= ... < 7 years,4,male   : single,none,3,building society savings agreement/ life insurance,32,none,own,1,skilled employee / official,1,none,no,1",
    "... <    0 DM,24,existing credits paid back duly till now,car (used),6579,... <  100 DM,unemployed,4,male   : single,none,2,unknown / no property,29,none,for free,1,management/ self-employed/ highly qualified employee/ officer,1,yes registered under the customers name,yes,1",
    "... <    0 DM,6,critical account/ other credits existing (not at this bank),furniture/equipment,1872,... <  100 DM,unemployed,4,male   : single,none,4,unknown / no property,36,none,for free,3,management/ self-employed/ highly qualified employee/ officer,1,yes registered under the customers name,yes,1",
    "... <    0 DM,12,all credits at this bank paid back duly,radio/television,2149,... <  100 DM,1  <= ... < 4 years,4,male   : divorced/separated,none,1,unknown / no property,29,none,for free,1,skilled employee / official,1,none,yes,2",
    "no checking account,18,critical account/ other credits existing (not at this bank),furniture/equipment,1817,... <  100 DM,1  <= ... < 4 years,4,female : divorced/separated/married,none,2,unknown / no property,28,none,own,2,skilled employee / official,1,none,yes,1",
    "0 <= ... <  200 DM,48,critical account/ other credits existing (not at this bank),furniture/equipment,5096,... <  100 DM,1  <= ... < 4 years,2,female : divorced/separated/married,none,3,car or other,30,none,own,1,management/ self-employed/ highly qualified employee/ officer,1,yes registered under the customers name,yes,2",
    "no checking account,42,existing credits paid back duly till now,radio/television,7166,unknown/ no savings account,4  <= ... < 7 years,2,male   : married/widowed,none,4,building society savings agreement/ life insurance,29,none,rent,1,skilled employee / official,1,yes registered under the customers name,yes,1",
    "no checking account,30,existing credits paid back duly till now,radio/television,2333,500 <= ... < 1000 DM,.. >= 7 years,4,male   : single,none,2,car or other,30,bank,own,1,management/ self-employed/ highly qualified employee/ officer,1,none,yes,1",
    "... <    0 DM,36,existing credits paid back duly till now,furniture/equipment,5179,... <  100 DM,4  <= ... < 7 years,4,male   : single,none,2,building society savings agreement/ life insurance,29,none,own,1,skilled employee / official,1,none,yes,2",
    "no checking account,12,critical account/ other credits existing (not at this bank),furniture/equipment,3331,... <  100 DM,.. >= 7 years,2,male   : single,none,4,building society savings agreement/ life insurance,42,stores,own,1,skilled employee / official,1,none,yes,1",
    "... >= 200 DM / salary assignments for at least 1 year,24,critical account/ other credits existing (not at this bank),radio/television,3148,unknown/ no savings account,1  <= ... < 4 years,3,male   : single,none,2,car or other,31,none,own,2,skilled employee / official,1,yes registered under the customers name,yes,1",
    "no checking account,24,delay in paying off in the past,business,2375,500 <= ... < 1000 DM,1  <= ... < 4 years,4,male   : single,none,2,car or other,44,none,own,2,skilled employee / official,2,yes registered under the customers name,yes,1",
    "no checking account,9,existing credits paid back duly till now,education,3832,unknown/ no savings account,.. >= 7 years,1,male   : single,none,4,real estate,64,none,own,1,unskilled - resident,1,none,yes,1",
    "no checking account,18,critical account/ other credits existing (not at this bank),furniture/equipment,1817,... <  100 DM,1  <= ... < 4 years,4,female : divorced/separated/married,none,2,unknown / no property,28,none,own,2,skilled employee / official,1,none,yes,1",
    "no checking account,9,existing credits paid back duly till now,business,1449,... <  100 DM,4  <= ... < 7 years,3,female : divorced/separated/married,none,2,car or other,27,none,own,2,skilled employee / official,1,none,yes,1"
]

In [ ]:
import json

documents = [] # letters
file_path = "/content/feature-matching-1.jsonl"

with open(file_path, 'r') as f:
    for line in f:
        try:
            data = json.loads(line)
            if 'text' in data:
                documents.append(data['text'])
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON line: {line}")

print(f"Loaded {len(documents)} strings from {file_path}")

Loaded 47 strings from /content/feature-matching-1.jsonl


In [ ]:
pip install gensim nltk pot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.8 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import gensim.downloader as api
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load a pre-trained Word2Vec model
# This might take a few minutes to download depending on the model size.
# 'glove-wiki-gigaword-50' is a relatively small model for demonstration.
print("Downloading pre-trained word embedding model...")
model = api.load("glove-wiki-gigaword-50")
print("Model loaded.")

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Tokenizes text, converts to lowercase, and removes stopwords and non-alphabetic tokens."""
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

# Ensure both lists have the same length for matching items
min_len = min(len(queries), len(documents))



Model loaded.


In [ ]:
def preprocess_documents_excluding_query_phrases(documents, queries, preprocess_func):
    processed_documents_filtered = []
    min_len = min(len(documents), len(queries))

    for i in range(min_len):
        doc_text = documents[i]
        query_text = queries[i]

        processed_doc_words = preprocess_func(doc_text)
        processed_query_words = preprocess_func(query_text)

        # Remove words from the document that are present in the query
        filtered_doc_words = [word for word in processed_doc_words if word not in processed_query_words]
        processed_documents_filtered.append(filtered_doc_words)

    # For any remaining documents beyond min_len, just preprocess them normally
    for i in range(min_len, len(documents)):
        processed_documents_filtered.append(preprocess_func(documents[i]))

    return processed_documents_filtered

# Apply the new preprocessing method
processed_documents_with_exclusion = preprocess_documents_excluding_query_phrases(documents, queries, preprocess_text)

print("Original document (first 20 words):\n", documents[0][:150], "...")
print("Original query:\n", queries[0])
print("\nProcessed document (first 10 words) excluding query phrases:\n", processed_documents_with_exclusion[0][:10])

print("\nOriginal document (first 20 words):\n", documents[1][:150], "...")
print("Original query:\n", queries[1])
print("\nProcessed document (first 10 words) excluding query phrases:\n", processed_documents_with_exclusion[1][:10])

Original document (first 20 words):
 Letter 1  
Berlin, 18 September 2025  

Dear Loan Officer,  

My name is Miguel Ramírez, a 35-year-old Spanish aircraft-maintenance supervisor employe ...
Original query:
 no checking account,12,critical account/ other credits existing (not at this bank),radio/television,976,unknown/ no savings account,.. >= 7 years,4,male   : single,none,4,car or other,35,none,own,2,skilled employee / official,1,none,yes,1

Processed document (first 10 words) excluding query phrases:
 ['letter', 'berlin', 'september', 'dear', 'loan', 'officer', 'name', 'miguel', 'ramírez', 'spanish']

Original document (first 20 words):
 **Letter 3:**

Dear Loan Officer,

I am writing to apply for a loan of $886 to upgrade my television. I am currently employed as a customer service re ...
Original query:
 no checking account,12,existing credits paid back duly till now,radio/television,886,unknown/ no savings account,1  <= ... < 4 years,4,female : divorced/separated/married,none,2,

In [ ]:
processed_documents_as_strings = []
for doc_words in processed_documents_with_exclusion:
    processed_documents_as_strings.append(' '.join(doc_words))

# Display the first few transformed documents to verify
for i in range(min(5, len(processed_documents_as_strings))):
    print(f"Processed Document {i}: {processed_documents_as_strings[i][:150]}...")


Processed Document 0: letter berlin september dear loan officer name miguel ramírez spanish supervisor employed flughafen berlin brandenburg gmbh past eight years respectfu...
Processed Document 1: letter dear loan officer writing apply loan upgrade television currently employed customer service representative telecommunications company worked pa...
Processed Document 2: letter augsburg september dear sir madam marco russo italian national residing augsburg since years old married one young daughter past six years work...
Processed Document 3: motivation letter dear maintenance worker local school earning monthly need perform duties efficiently current vehicle engine failed last month withou...
Processed Document 4: letter captain miriam schneider esteemed sir madam captain miriam schneider separated commissioned officer federal police salary grade stationed hambu...


In [ ]:
print("\nCalculating Earth Mover's Distance (WMD) for matching query-document pairs...")
for i in range(min_len):
    query_text = queries[i]
    document_text = processed_documents_as_strings[i] #documents[i]

    processed_query = preprocess_text(query_text)
    print(processed_query)
    processed_document = preprocess_text(document_text)

    # Filter out words not present in the model's vocabulary
    processed_query_in_vocab = [word for word in processed_query if word in model.key_to_index]
    processed_document_in_vocab = [word for word in processed_document if word in model.key_to_index]

    if not processed_query_in_vocab or not processed_document_in_vocab:
        distance = float('inf') # Or handle as appropriate if one text is empty after filtering
        print(f"Pair {i}: Cannot compute WMD. One or both texts have no words in vocabulary after preprocessing.")
    else:
        # Compute Word Mover's Distance
        distance = model.wmdistance(processed_query_in_vocab, processed_document_in_vocab)
        print(f"WMD between query {i} and document {i}: {distance:.4f}")


Calculating Earth Mover's Distance (WMD) for matching query-document pairs...
['checking', 'critical', 'credits', 'existing', 'bank', 'savings', 'account', 'male', 'single', 'car', 'none', 'skilled', 'employee', 'none']
WMD between query 0 and document 0: 0.9078
['checking', 'existing', 'credits', 'paid', 'back', 'duly', 'till', 'savings', 'female', 'car', 'none', 'skilled', 'employee', 'none']
WMD between query 1 and document 1: 0.8859
['existing', 'credits', 'paid', 'back', 'duly', 'till', 'male', 'real', 'bank', 'unskilled', 'none']
WMD between query 2 and document 2: 0.9631
['critical', 'credits', 'existing', 'bank', 'car', 'used', 'dm', 'female', 'unknown', 'stores', 'unskilled', 'none']
WMD between query 3 and document 3: 0.9357
['checking', 'critical', 'credits', 'existing', 'bank', 'car', 'used', 'dm', 'female', 'car', 'none', 'highly', 'qualified', 'yes', 'registered', 'customers', 'name']
WMD between query 4 and document 4: 0.9430
['checking', 'delay', 'paying', 'past', 'mal

Calculating Earth Mover's Distance (WMD) for matching query-document pairs...
WMD between query 0 and document 0: 0.8910
WMD between query 1 and document 1: 0.8220
WMD between query 2 and document 2: 0.9546
WMD between query 3 and document 3: 0.8428
WMD between query 4 and document 4: 0.9197
WMD between query 5 and document 5: 0.8620
WMD between query 6 and document 6: 0.8496
WMD between query 7 and document 7: 0.9498
WMD between query 8 and document 8: 0.8415
WMD between query 9 and document 9: 0.8540
WMD between query 10 and document 10: 0.8845
WMD between query 11 and document 11: 0.8467
WMD between query 12 and document 12: 0.7283
WMD between query 13 and document 13: 0.8926
WMD between query 14 and document 14: 0.8687
WMD between query 15 and document 15: 0.7972
WMD between query 16 and document 16: 0.8713
WMD between query 17 and document 17: 0.8596
WMD between query 18 and document 18: 0.7717
WMD between query 19 and document 19: 0.8617
WMD between query 20 and document 20: 0.9052
WMD between query 21 and document 21: 0.9245
WMD between query 22 and document 22: 0.9164
WMD between query 23 and document 23: 0.8402
WMD between query 24 and document 24: 0.8843
WMD between query 25 and document 25: 0.8489
WMD between query 26 and document 26: 0.8363
WMD between query 27 and document 27: 0.8599
WMD between query 28 and document 28: 0.8528
WMD between query 29 and document 29: 0.8293
WMD between query 30 and document 30: 0.8833
WMD between query 31 and document 31: 0.8182
WMD between query 32 and document 32: 0.8692
WMD between query 33 and document 33: 0.8267
WMD between query 34 and document 34: 0.9063
WMD between query 35 and document 35: 0.7918
WMD between query 36 and document 36: 0.7901
WMD between query 37 and document 37: 0.8882
WMD between query 38 and document 38: 0.8879
WMD between query 39 and document 39: 0.9061
WMD between query 40 and document 40: 0.8665
WMD between query 41 and document 41: 0.8616
WMD between query 42 and document 42: 0.8329
WMD between query 43 and document 43: 0.8000
WMD between query 44 and document 44: 0.9134
WMD between query 45 and document 45: 0.8636

In [ ]:
print(f"Processed Query 10: {processed_query[10]}...")
print(f"Processed Document 10: {processed_documents_as_strings[10]}...")

Processed Query 10: skilled...
Processed Document 10: dear madam sir write kwame asante ghanaian precision welder employed siemens energy berlin past years request acquire welding station start mobile repair service local farmers salary rent support daughter available installment equals amount two prior repaid time checking overdraft savings book shows resided neukölln flat four years mercedes carry debt equipment supplier reserved unit approval ensure prompt repayment combined earnings gratitude kwame asante...


In [ ]:
def compute_wmd_for_pairs(texts, model, preprocess_func):
    print("\nCalculating Earth Mover's Distance (WMD) for all unique pairs...")
    processed_texts = [preprocess_func(text) for text in texts]

    results = []
    for i in range(len(processed_texts)):
        for j in range(i + 1, len(processed_texts)): # Iterate through unique pairs
            processed_text1_in_vocab = [word for word in processed_texts[i] if word in model.key_to_index]
            processed_text2_in_vocab = [word for word in processed_texts[j] if word in model.key_to_index]

            if not processed_text1_in_vocab or not processed_text2_in_vocab:
                distance = float('inf')
                status = "Cannot compute WMD. One or both texts have no words in vocabulary after preprocessing."
            else:
                distance = model.wmdistance(processed_text1_in_vocab, processed_text2_in_vocab)
                status = f"WMD between text {i} and text {j}: {distance:.4f}"
            print(status)
            results.append(((i, j), distance))
    return results

# Compute WMD for all pairs within the 'queries' list
query_pair_wmd_results = compute_wmd_for_pairs(queries, model, preprocess_text)


Calculating Earth Mover's Distance (WMD) for all unique pairs...
WMD between text 0 and text 1: 0.3125
WMD between text 0 and text 2: 0.5911
WMD between text 0 and text 3: 0.4855
WMD between text 0 and text 4: 0.4933
WMD between text 0 and text 5: 0.4902
WMD between text 0 and text 6: 0.5788
WMD between text 0 and text 7: 0.4331
WMD between text 0 and text 8: 0.3458
WMD between text 0 and text 9: 0.5894
WMD between text 0 and text 10: 0.4488
WMD between text 0 and text 11: 0.5993
WMD between text 0 and text 12: 0.4763
WMD between text 0 and text 13: 0.4595
WMD between text 0 and text 14: 0.4003
WMD between text 0 and text 15: 0.1331
WMD between text 0 and text 16: 0.2103
WMD between text 0 and text 17: 0.5277
WMD between text 0 and text 18: 0.3276
WMD between text 0 and text 19: 0.2856
WMD between text 0 and text 20: 0.4569
WMD between text 0 and text 21: 0.4304
WMD between text 0 and text 22: 0.4679
WMD between text 0 and text 23: 0.6756
WMD between text 0 and text 24: 0.5627
WMD bet